In [1]:
import cv2
import os
import pytesseract
from fuzzywuzzy import fuzz

import download
from extract import unsharp, process_text
import validations

In [2]:
estimations = {}

if os.path.isfile("estimations.txt"):
    f = open("estimations.txt", "r")
    lines = f.readlines()
    lines = [l.split(' ') for l in lines]
    estimations = {l[0]: float(l[1]) for l in lines}
    f.close()

In [3]:
#filters = ['h_600', 'h_800', '*1', '*2', '*3', '*4', '*5', '*6', 'gray', 'unsharp', 'thresh', 'blur', '']
filters = ['h_600', 'h_800', '*1', '*2', '*3', 'gray', 'unsharp', 'thresh', 'blur', '']
filters = [f if len(f) == 0 else f+':' for f in filters]
def is_scale(p):
    return p.startswith('h_') or p.startswith('*')

In [4]:
params = []
pset = set()
for f1 in filters:
    for f2 in filters:
        if f2 != '' and f2 == f1: continue
        if is_scale(f1) and is_scale(f2): continue
        for f3 in filters:
            if f3 == f1 or f3 == f2: continue
            if is_scale(f1) and is_scale(f3): continue
            if is_scale(f2) and is_scale(f3): continue
            val = f1+f2+f3
            if val in pset: continue
            pset.add(val)
            params.append(val)

In [5]:
len(params)

265

In [6]:
def estimate(ps, video_id, frame, text):
    key = "%s:%s:%f" % (ps, video_id, frame)
    if key in estimations:
        return estimations[key]
    
    im = download.get_frame(video_id, frame)
    for p in ps.split(':'):
        if len(p) == 0: continue
        if p.startswith('h_'):
            target_height = int(p[2:])
            h = im.shape[0]
            w = im.shape[1]
            scale = target_height / h
            im = cv2.resize(im, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
        elif p.startswith('*'):
            scale = int(p[1:])
            im = cv2.resize(im, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
        elif p == 'gray':
            im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        elif p == 'unsharp':
            im = unsharp(im)
        elif p == 'thresh':
            im = cv2.threshold(im, 150, 255, cv2.THRESH_BINARY)[1]
        elif p == 'blur':
            im = cv2.medianBlur(im, 3)
    tess_text = pytesseract.image_to_string(im)
    tess_text = process_text(tess_text)
    score = fuzz.ratio(text.lower(), tess_text.lower())
    
    f = open("estimations.txt", "a")
    f.write("%s %f\n" % (key, score))
    f.close()
    
    return score

In [7]:
best = None
best_score = 0
rate_list = []

for p in params:
    total = 0
    score_min = 101*len(validations.data)
    score_max = -1
    for d in validations.data:
        score = estimate(p, **d)
        if score > 0:
            total += score
        score_min = min(score_min, score)
        score_max = max(score_max, score)
    total = total // len(validations.data)
    print(p, total, score_min, score_max)
    if total > best_score:
        best = p
        best_score = total
    if total > 0:
        rate_list.append((p, total, score_min, score_max))

h_600:gray:unsharp: 79.0 0.0 99
h_600:gray:thresh: 85.0 64.0 99
h_600:gray:blur: 91.0 72 99
h_600:gray: 90.0 76.0 99
h_600:unsharp:gray: 78.0 0.0 96.0
h_600:unsharp:thresh: 16.0 0.0 34
h_600:unsharp:blur: 91.0 78.0 99
h_600:unsharp: 42 0.0 95
h_600:thresh:gray: 84.0 65 96
h_600:thresh:unsharp: 4 0.0 28
h_600:thresh:blur: 12 0.0 28
h_600:thresh: 4 0.0 28
h_600:blur:gray: 88.0 75 99
h_600:blur:unsharp: 74.0 0.0 98
h_600:blur:thresh: 12 0.0 28
h_600:blur: 93.0 76.0 99
h_800:gray:unsharp: 88.0 69 95
h_800:gray:thresh: 91.0 83 97
h_800:gray:blur: 90.0 75 99.0
h_800:gray: 74.0 0.0 98
h_800:unsharp:gray: 89.0 68 95
h_800:unsharp:thresh: 19.0 0.0 52
h_800:unsharp:blur: 85.0 68 97
h_800:unsharp: 73.0 0 95
h_800:thresh:gray: 86.0 70.0 99
h_800:thresh:unsharp: 25.0 0.0 76
h_800:thresh:blur: 6 0.0 20
h_800:thresh: 25.0 0.0 76
h_800:blur:gray: 90.0 74 99.0
h_800:blur:unsharp: 88.0 67 98
h_800:blur:thresh: 6 0.0 20
h_800:blur: 87.0 72.0 98
*1:gray:unsharp: 76.0 0.0 99
*1:gray:thresh: 72.0 0.0 99
*1:

Top best avg score

In [8]:
sorted(rate_list, key=lambda r: -r[1])[:10]

[('gray:h_800:thresh:', 95.0, 90.0, 99),
 ('unsharp:h_600:gray:', 95.0, 91.0, 99),
 ('unsharp:h_800:gray:', 95.0, 93, 99.0),
 ('h_800:', 95.0, 91.0, 98),
 ('*1:', 95.0, 90.0, 98),
 ('*1:gray:', 94.0, 86.0, 99),
 ('*2:unsharp:blur:', 94.0, 90.0, 97),
 ('gray:h_800:blur:', 94.0, 87, 98.0),
 ('gray:*1:', 94.0, 86.0, 99),
 ('gray:*2:', 94.0, 90.0, 96.0)]

Top best min score

In [9]:
sorted(rate_list, key=lambda r: (-r[2], -r[1]))[:10]

[('unsharp:h_800:gray:', 95.0, 93, 99.0),
 ('unsharp:h_600:gray:', 95.0, 91.0, 99),
 ('h_800:', 95.0, 91.0, 98),
 ('gray:h_800:thresh:', 95.0, 90.0, 99),
 ('*1:', 95.0, 90.0, 98),
 ('*2:unsharp:blur:', 94.0, 90.0, 97),
 ('gray:*2:', 94.0, 90.0, 96.0),
 ('*3:gray:', 92.0, 89, 96.0),
 ('*3:blur:gray:', 91.0, 89, 95.0),
 ('gray:*3:blur:', 92.0, 88, 96.0)]